## Worldometer Site Web Scrapping

---

In [1]:
# Import libraries

import os
import re
import glob
import requests 
import numpy as np
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup

In [2]:
# Import dataset

link = 'https://www.worldometers.info/coronavirus/'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# Find table

thead = soup.find_all('thead')[-1]
head = thead.find_all('tr')
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

In [4]:
# Get the table contents

head_rows = []
body_rows = []

for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
print(head_rows)

[['#', 'Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical', 'Tot\xa0Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/\n1M pop\n', 'Population', 'Continent', '1 Caseevery X ppl', '1 Deathevery X ppl', '1 Testevery X ppl']]


In [5]:
# Save content in dataframe

df_wm = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

df_wm.tail(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
217,210,Falkland Islands,16,,,,13,,3,,"4,544",,"4,245","1,205,623","3,521",South America,220,,1
218,211,Saint Pierre Miquelon,16,,,,12,,4,,"2,767",,"2,814","486,683","5,782",North America,361,,2
219,212,Montserrat,13,,1,,12,,0,,"2,604",200,577,"115,562","4,993",North America,384,"4,993",9
220,213,Western Sahara,10,,1,,8,,1,,17,2,,,"603,351",Africa,"60,335","603,351",
221,214,MS Zaandam,9,,2,,,,7,,,,,,,,,,


In [6]:
# Drop unwanted rows & columns

df_wm = df_wm.iloc[8:, :-3].reset_index(drop=True)
df_wm = df_wm.drop('#', axis=1)
df_wm.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent
0,USA,"13,919,870",,"274,332",,"8,222,879",,"5,422,659","25,307","41,951",827,"194,908,749","587,407","331,812,157",North America
1,India,"9,463,254",,"137,659",,"8,888,595",,"437,000","8,944","6,830",99,"140,379,976","101,310","1,385,641,672",Asia
2,Brazil,"6,336,278",,"173,165",,"5,601,804",,"561,309","8,318","29,721",812,"21,900,000","102,726","213,188,844",South America
3,Russia,"2,295,654",,"39,895",,"1,778,704",,"477,055","2,300","15,728",273,"76,422,849","523,585","145,960,689",Europe
4,France,"2,222,488",,"52,731",,"162,281",,"2,007,476","3,751","34,017",807,"20,618,607","315,588","65,333,951",Europe


In [7]:
# Rename & rearrange columns
df_wm.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']

df_wm = df_wm[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]

df_wm.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"331,812,157","13,919,870",,"274,332",,"8,222,879",,"5,422,659","25,307","41,951",827,"194,908,749","587,407"
1,India,Asia,"1,385,641,672","9,463,254",,"137,659",,"8,888,595",,"437,000","8,944","6,830",99,"140,379,976","101,310"
2,Brazil,South America,"213,188,844","6,336,278",,"173,165",,"5,601,804",,"561,309","8,318","29,721",812,"21,900,000","102,726"
3,Russia,Europe,"145,960,689","2,295,654",,"39,895",,"1,778,704",,"477,055","2,300","15,728",273,"76,422,849","523,585"
4,France,Europe,"65,333,951","2,222,488",,"52,731",,"162,281",,"2,007,476","3,751","34,017",807,"20,618,607","315,588"


In [8]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [9]:
# Addding 'WHO Region' & list of countries with missing "WHO Region"

df_wm['WHO Region'] = df_wm['Country/Region'].map(who_region)
df_wm[df_wm['WHO Region'].isna()]['Country/Region'].unique()

array(['French Polynesia', 'French Guiana', 'Guadeloupe', 'Réunion',
       'Martinique', 'Mayotte', 'CAR', 'Aruba', 'Curaçao',
       'Channel Islands', 'Sint Maarten', 'Gibraltar', 'Turks and Caicos',
       'Saint Martin', 'Diamond Princess', 'Faeroe Islands',
       'Isle of Man', 'Cayman Islands', 'Caribbean Netherlands',
       'Brunei ', 'St. Barth', 'St. Vincent Grenadines',
       'British Virgin Islands', 'Macao', 'New Caledonia',
       'Falkland Islands', 'Saint Pierre Miquelon', 'Montserrat',
       'MS Zaandam'], dtype=object)

In [10]:
# Data corrections

for col in df_wm.columns[2:]:
    # replace comma with empty string
    df_wm[col] = df_wm[col].str.replace('[,+ ]', '', regex=True)
    # replace 'N/A' with empty string
    df_wm[col] = df_wm[col].str.replace('N/A', '', regex=False)

# replace empty strings with np.nan
df_wm = df_wm.replace('', np.nan)

df_wm.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,331812157,13919870,NaN,274332,NaN,8222879,NaN,5422659,25307,41951,827,194908749,587407,Americas
1,India,Asia,1385641672,9463254,NaN,137659,NaN,8888595,NaN,437000,8944,6830,99,140379976,101310,South-EastAsia
2,Brazil,South America,213188844,6336278,NaN,173165,NaN,5601804,NaN,561309,8318,29721,812,21900000,102726,Americas
3,Russia,Europe,145960689,2295654,NaN,39895,NaN,1778704,NaN,477055,2300,15728,273,76422849,523585,Europe
4,France,Europe,65333951,2222488,NaN,52731,NaN,162281,NaN,2007476,3751,34017,807,20618607,315588,Europe


In [11]:
# Save as "csv" file

df_wm.to_csv('worldometer_data.csv', index=False)